# Variational Quantum Eigensolver



A common application of the Variational Quantum Eigensolver (VQE) algorithm is to compute the ground state energy of a molecular system. The code below demonstrates how to perform classical preprocessing for a $H_2$ molecule (i.e. obtain the integrals from a Hartree-Fock computation to build the molecular Hamiltonian), prepare the initial Hartree-Fock state on the quantum register, add the parameterized UCCSD ansatz to the kernel, and select the COBYLA optimizer.  We are then ready to call `cudaq:vqe` to estimate the minimum energy of the system.

In [ ]:
%pip install openfermionpyscf

In [4]:
import cudaq
import cudaq.lib

geometry = [('H', (0., 0., 0.)), ('H', (0., 0., .7474))]
molecule, data = cudaq.chemistry.create_molecular_hamiltonian(
    geometry, 'sto-3g', 1, 0)

electron_count = data.n_electrons
qubit_count = 2 * data.n_orbitals

@cudaq.kernel(jit=True)
def kernel(qubit_count: int, angles:list[float]):
    qubits = cudaq.qvector(qubit_count)

    # Prepare the Hartree Fock State.
    x(qubits[0])
    x(qubits[1])

    # Adds parameterized gates based on the UCCSD ansatz.
    chemistry.uccsd(qubits, angles, 2, qubit_count)

parameter_count = cudaq.lib.chemistry.uccsd_num_parameters(electron_count, qubit_count)
optimizer = cudaq.optimizers.COBYLA()
energy, parameters = cudaq.vqe(kernel,
                               molecule,
                               optimizer,
                               parameter_count=parameter_count)

print(energy)

-1.1371755939165522


We see that the energy has converged to the ground state minimum for $H_{2}$, as expected.